In [1]:
import torch

print("torch version:", torch.__version__)

# Check gpu availability


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.zeros(1).cuda())


torch version: 2.1.2+cu121
cpu


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [3]:
# Check if the model can be loaded
from nrms import NRMSModel
from hyperparameters import HyperParameters

# hparams contains the hyperparameters of the model:
# They are:
# vocab_size: int
# word_embedding_dim: int
# dropout: float


hparams = HyperParameters(
    vocab_size=100, 
    word_embedding_dim=300, 
    dropout=0.1, 
    head_num=8,
    head_dim=64,
    attention_hidden_dim=512,
    title_size=20
)

word2vec_embedding = torch.randn(hparams.vocab_size, hparams.word_embedding_dim)

model = NRMSModel(hparams=hparams, word2vec_embedding=word2vec_embedding)

print(model)




NRMSModel(
  (word2vec_embedding): Embedding(100, 300)
  (dropout): Dropout(p=0.1, inplace=False)
  (news_encoder): Sequential(
    (0): Embedding(100, 300)
    (1): Dropout(p=0.1, inplace=False)
    (2): SelfAttention(
      (query_layer): Linear(in_features=64, out_features=512, bias=True)
      (key_layer): Linear(in_features=64, out_features=512, bias=True)
      (value_layer): Linear(in_features=64, out_features=512, bias=True)
    )
    (3): Dropout(p=0.1, inplace=False)
    (4): AttentionLayer2(
      (attention_weight): Linear(in_features=512, out_features=1, bias=True)
    )
  )
  (user_encoder): Sequential(
    (0): Linear(in_features=20, out_features=512, bias=True)
    (1): SelfAttention(
      (query_layer): Linear(in_features=64, out_features=512, bias=True)
      (key_layer): Linear(in_features=64, out_features=512, bias=True)
      (value_layer): Linear(in_features=64, out_features=512, bias=True)
    )
    (2): AttentionLayer2(
      (attention_weight): Linear(in_featu

TODO HERFRA OG NED

In [25]:
import os
import sys
from pathlib import Path
workspace_path = os.path.expandvars('$BLACKHOLE')
sys.path.append(workspace_path+'/DeepLearning/02456_news_project/src')
DATAPATH = Path(workspace_path+"/DeepLearning/ebnerd_data").expanduser()

from pathlib import Path
import polars as pl

class NewsDataset:
    def __init__(self, dataset_path: Path):

        self.dataset_path = dataset_path
        self.df_behaviors_train = self._load_behaviors_train()
        self.df_history_train = self._load_history_train()
        self.df_behaviors_validation = self._load_behaviors_validation()
        self.df_history_validation = self._load_history_validation()
        self.df_articles = self._load_articles()

        # Call the method to test data loading
        self.test_data_loading()


    def _load_behaviors_train(self):
        return pl.read_parquet(self.dataset_path.joinpath("train/behaviors.parquet"))

    def _load_history_train(self):
        return pl.read_parquet(self.dataset_path.joinpath("train/history.parquet"))

    def _load_behaviors_validation(self):
        return pl.read_parquet(self.dataset_path.joinpath("validation/behaviors.parquet"))

    def _load_history_validation(self):
        return pl.read_parquet(self.dataset_path.joinpath("validation/history.parquet"))

    def _load_articles(self):
        return pl.read_parquet(self.dataset_path.joinpath("articles.parquet"))

    def test_data_loading(self):
        # Print basic info about the loaded dataframes
        print("Train Behaviors DataFrame:")
        print(self.df_behaviors_train.shape)
        print(self.df_behaviors_train.head())

        print("\nTrain History DataFrame:")
        print(self.df_history_train.shape)
        print(self.df_history_train.head())

        print("\nValidation Behaviors DataFrame:")
        print(self.df_behaviors_validation.shape)
        print(self.df_behaviors_validation.head())

        print("\nValidation History DataFrame:")
        print(self.df_history_validation.shape)
        print(self.df_history_validation.head())

        print("\nArticles DataFrame:")
        print(self.df_articles.shape)
        print(self.df_articles.head())

# test
dataset = NewsDataset(dataset_path=DATAPATH.joinpath("ebnerd_small"))


Train Behaviors DataFrame:
(232887, 17)
shape: (5, 17)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ impressio ┆ article_i ┆ impressio ┆ read_time ┆ … ┆ is_subscr ┆ session_i ┆ next_read ┆ next_scr │
│ n_id      ┆ d         ┆ n_time    ┆ ---       ┆   ┆ iber      ┆ d         ┆ _time     ┆ oll_perc │
│ ---       ┆ ---       ┆ ---       ┆ f32       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ entage   │
│ u32       ┆ i32       ┆ datetime[ ┆           ┆   ┆ bool      ┆ u32       ┆ f32       ┆ ---      │
│           ┆           ┆ μs]       ┆           ┆   ┆           ┆           ┆           ┆ f32      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 149474    ┆ null      ┆ 2023-05-2 ┆ 13.0      ┆ … ┆ false     ┆ 759       ┆ 7.0       ┆ 22.0     │
│           ┆           ┆ 4         ┆           ┆   ┆           ┆           ┆           ┆          │
│           ┆           ┆ 07:47:53  

In [ ]:
# Use the dataloader to separate the data into batches / validation / test sets
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

for batch in dataloader:
    print(batch)
    break


In [ ]:
# Train the model

from train import train

train(
    model=model,
    dataloader=dataloader,
    hparams=hparams,
    device=device
)

# Save the model
torch.save(model.state_dict(), "model.pth")